In [ ]:
import requests
import json
import re
import os

git_fine_grained_personal_token = os.getenv("my_git_key")

# Assert that the environment variable is defined
assert (
    git_fine_grained_personal_token is not None
), "my_git_key environment variable is not defined!"


headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": "token " + git_fine_grained_personal_token,
}


def get_issue_count(owner: str, repo: str, state: str):
    url = f"https://api.github.com/search/issues?q=is:issue+is:{state}+repo:{owner}/{repo}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data["total_count"] if "total_count" in data else 0
    else:
        print(f"Error fetching issues: {response.status_code} - {response.text}")
        return None


def get_repo_state(owner: str, repo: str):
    response = requests.get(
        f"https://api.github.com/repos/{owner}/{repo}", headers=headers
    )
    repo_info = response.json()
    star_count = repo_info["stargazers_count"]

    # Fetch the last commit time
    response = requests.get(
        f"https://api.github.com/repos/{owner}/{repo}/commits", headers=headers
    )
    commits = response.json()
    last_commit_time = commits[0]["commit"]["committer"][
        "date"
    ]  # Assuming sorted by date

    return {
        "Url": f"https://github.com/{owner}/{repo}",
        "Open Issues": get_issue_count(owner, repo, "open"),
        "Closed Issues": get_issue_count(owner, repo, "closed"),
        "Star Count": star_count,
        "Last Commit Time": last_commit_time,
    }


def get_repo_state_from_url(url: str):
    # (?:/|$) Non-capturing group that matches either "end of line or nearest /
    m = re.search("https://github.com/(.*?)/(.*?)(?:/|$)", url)
    if not m:
        return {"Url": url}
    else:
        owner, repo = m.groups()
        return get_repo_state(owner, repo)


def parse_lines_of_markdown_links(raw_text: str):
    for line in raw_text.split("\n"):
        m = re.search("\[.*?\]\((.*?)\)", line)
        if not m:
            print(line)
        else:
            url = m.group(1)
            repo_stats = get_repo_state_from_url(url)
            print(json.dumps(repo_stats, indent=4))

In [ ]:
owner = "apache"
repo = "brpc"
result = get_repo_state(owner, repo)
print(json.dumps(result, indent=4))

In [ ]:
parse_lines_of_markdown_links(
    """* [Apache Thrift](https://thrift.apache.org/) - Efficient cross-language IPC/RPC, works between C++, Java, Python, PHP, C#, and many more other languages. Originally developed by Facebook. [Apache2]
* [Boost.Interprocess](https://github.com/boostorg/interprocess) - Header-only Boost library that supports kernel-level shared memory and memory-mapped files, with in-built synchronization mechanisms (semaphores, mutexes, and more). [Boost] [website](https://boost.org/libs/interprocess)
* [bRPC](https://github.com/apache/brpc) - bRPC is an Industrial-grade RPC framework using C++ Language, which is often used in high performance system such as Search, Storage, Machine learning, Advertisement, Recommendation etc. [Apache2] [website](https://brpc.apache.org/)
* [Cap'n Proto](https://github.com/capnproto/capnproto) - Fast data interchange format and capability-based RPC system. [MIT] [website](https://capnproto.org/)
* [eCAL](https://github.com/continental/ecal) - Pub/sub, client/server, C++/Python/C#, various message protocols (protobuf, capnproto ..). [Apache2] [website](http://www.ecal.io/)
* [gRPC](https://github.com/grpc/grpc) - A high performance, open source, general-purpose RPC framework. [BSD] [website](http://www.grpc.io/)
* [Ice](https://github.com/zeroc-ice/ice) - Comprehensive RPC framework with support for C++, C#, Java, JavaScript, Python and more. [GPLv2]
* [iceoryx](https://github.com/eclipse-iceoryx/iceoryx) - True zero-copy inter-process communication framework for safety critical systems with bindings for C, Rust. Runs on Linux, QNX, Windows, Mac OS, FreeBSD. [Apache2] [website](https://iceoryx.io/)
* [libjson-rpc-cpp](https://github.com/cinemast/libjson-rpc-cpp) - JSON-RPC framework for C++ servers and clients. [MIT]
* [nanomsg](https://github.com/nanomsg/nanomsg) - A simple high-performance implementation of several "scalability protocols". [MIT] [website](http://nanomsg.org/)
* [nng](https://github.com/nanomsg/nng) - nanomsg-next-generation, a light-weight brokerless messaging library. [MIT] [website](https://nanomsg.github.io/nng/)
* [rpclib](https://github.com/rpclib/rpclib) - A modern C++ msgpack-RPC server and client library. [MIT]
* [simple-rpc-cpp](https://github.com/pearu/simple-rpc-cpp) - A simple RPC wrapper generator to C/C++ functions. [BSD]
* [SRPC](https://github.com/sogou/srpc) - A lightweight RPC system that supports multiple protocols and OpenTelemetry. [Apache2]
* [WAMP](http://wamp.ws/) - Provides RPC and pub/sub messaging patterns. (various implementations, various languages)
* [xmlrpc-c](http://xmlrpc-c.sourceforge.net/) - A lightweight RPC library based on XML and HTTP. [BSD]
"""
)